In [6]:
import sys
import os
import pytz
from datetime import datetime

import psycopg2
import pandas as pd
import numpy as np

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))
from constants import itemtype, currency, league

FILENAME_SUFFIX = '_1c'
league_id = league.BREACH
MIN_PRICE = 1  # Anything below this (in chaos) is considered worthless
FRESH_SECONDS = 7*24*60*60  # Assume undercut items worth MIN_PRICE will be bought in this time

In [7]:
rates = currency.get_exchange_rates(league.get_name(league_id))
print(rates)

def get_chaos_price(price, currency):
    return price * rates[currency]

def is_item_valuable(price, currency, added_time, sold_time):
    """
    Checks if an item is valuable or not.
    May also return None if we can't tell.
    """
    sold = sold_time > datetime(2000, 1, 1, tzinfo=pytz.UTC)
    price = get_chaos_price(price, currency)

    # If item was sold at min price or higher, it's valuable
    if price >= MIN_PRICE and sold:
        return True

    return False

def is_item_worthless(price, currency, added_time, sold_time):
    price = get_chaos_price(price, currency)
    fresh = (datetime.now(tz=pytz.UTC) - added_time).total_seconds() < FRESH_SECONDS

    # If item was offered for a low price for more than the fresh time, assume it's worthless
    if price < MIN_PRICE and not fresh:
        return True

    # If item is offered for a low price but the offer is younger than a day, ignore it

    # If item is sold for less than min price, ignore it 
    # because we don't know if the buyer would have also bought it at a higher price
        
    return False


dbconn = psycopg2.connect(dbname='poeria')
db = dbconn.cursor()

{4: 1.0}


# Rings

In [8]:
db.execute("""
SELECT Price, Currency, AddedTime, SoldTime, -- 3
       Corrupted, Sockets, DoubledInBreach,  -- 6
       Strength, Dexterity, Intelligence,    -- 9
       Life, Mana, Evasion, EnergyShield,    -- 13
       Accuracy, CritChance, AttackSpeed, CastSpeed,   -- 17
       FireResist, ColdResist, LightningResist, ChaosResist,  -- 21
       AddedPhysAttackDamage, AddedFireAttackDamage, AddedColdAttackDamage,  --24
       AddedLightningAttackDamage, AddedChaosAttackDamage,  -- 26                                                  -- 31
       IncreasedEleDamage, IncreasedWeaponEleDamage, IncreasedFireDamage, IncreasedColdDamage, IncreasedLightningDamage, 
       ItemRarity, LifeLeech, ManaLeech, LifeGainOnHit, LifeGainOnKill, LifeRegen, ManaGainOnKill, ManaRegen,   -- 39
       AvoidFreeze, GrantedSkillId, ManaGainOnHit, DamageToMana, LightRadius  --44
  FROM StashContents s, RingItems r
 WHERE s.league = %s
   AND s.ItemId = r.ItemId""",
           (league_id,))

features = []
targets = []

num_valuable = 0
num_worthless = 0

for row in db:
    valuable = is_item_valuable(row[0], row[1], row[2], row[3])
    worthless = is_item_worthless(row[0], row[1], row[2], row[3])
    
    # Ignore this item if we can't tell if it's valuable or not
    if not valuable and not worthless:
        continue
        
    features.append({
        'Price': get_chaos_price(row[0], row[1]) if valuable else 0,
        'Corrupted': row[4],
        'HasSocket': len(row[5]),
        'HasWhiteSocket': 1 if row[5] == 'W' else 0, 
        'DoubledInBreach': row[6],  
        'Strength': row[7],  
        'Dexterity': row[8],  
        'Intelligence': row[9],  
        'Life': row[10],  
        'Mana': row[11],  
        'Evasion': row[12],  
        'EnergyShield': row[13],  
        'Accuracy': row[14],  
        'CritChance': row[15],  
        'AttackSpeed': row[16],  
        'CastSpeed': row[17],  
        'TotalEleResist': row[18] + row[19] + row[20],  
        'ChaosResist': row[21],  
        'AddedPhysAttackDamage': row[22],  
        'AddedFireAttackDamage': row[23],  
        'AddedColdAttackDamage': row[24],  
        'AddedLightningAttackDamage': row[25],  
        'AddedChaosAttackDamage': row[26],  
        'IncreasedEleDamage': row[27],  
        'IncreasedWeaponEleDamage': row[28],  
        'IncreasedFireDamage': row[29],  
        'IncreasedColdDamage': row[30],  
        'IncreasedLightningDamage': row[31],  
        'ItemRarity': row[32],  
        'LifeLeech': row[33],  
        'ManaLeech': row[34],  
        'LifeGainOnHit': row[35],  
        'LifeGainOnKill': row[36],  
        'LifeRegen': row[37],  
        'ManaGainOnKill': row[38],  
        'ManaRegen': row[39],  
        'ManaGainOnHit': row[40],  
        'DamageToMana': row[41],  
        'LightRadius': row[42],   
    })
    
    num_valuable += 1 if valuable else 0
    num_worthless += 1 if worthless else 0

print("Found a total of {} interesting items".format(len(features)))
print("    {:10} valuable ({:.1f}%)".format(num_valuable, 100*(num_valuable/len(features))))
print("    {:10} worthless ({:.1f}%)".format(num_worthless, 100*(num_worthless/len(features))))
print()

features = pd.DataFrame(features, dtype=int)
features.to_csv('ring_features{}.csv.bz2'.format(FILENAME_SUFFIX), compression='bz2')


KeyError: 6